In [1]:
// Spark Session başlatılır

import org.apache.spark.sql.{DataFrame, SparkSession}

val spark: SparkSession = SparkSession
      .builder()
      .appName("Read and Write operations with CSV files")
      .master("local")
      .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.152:4040
SparkContext available as 'sc' (version = 3.2.0, master = local[*], app id = local-1643562649392)
SparkSession available as 'spark'


import org.apache.spark.sql.{DataFrame, SparkSession}
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5964129f


In [4]:
val base_path: String = "/home/cevat/workspace/my-apache-spark-notes/scala/projects/read_write_csv/src/csv"

base_path: String = /home/cevat/workspace/my-apache-spark-notes/scala/projects/read_write_csv/src/csv


In [5]:
// Read Single CSV
val single_csv_path: String = base_path + "/test1.csv"
val first_csv_df: DataFrame = spark.read.csv(path = single_csv_path)
first_csv_df.show()

// CSV dosyası okunda ve bir DataFrame nesnesine dönüştürüldü.

+-------+----------+---+---------+-------+
|    _c0|       _c1|_c2|      _c3|    _c4|
+-------+----------+---+---------+-------+
|   name|   surname|age|     city|country|
|  Cevat|   Armutlu| 26| Istanbul| Turkey|
|   John|      Eric| 56| New York|America|
|Natalia|Khrushchev|  6|Moskovitz|Russiaa|
+-------+----------+---+---------+-------+



single_csv_path: String = /home/cevat/workspace/my-apache-spark-notes/scala/projects/read_write_csv/src/csv/test1.csv
first_csv_df: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 3 more fields]


In [6]:
// Read Multiple CSV: Folder
val csv_folder: String = base_path
val read_csv_folder: DataFrame = spark.read.csv(path = csv_folder)
// Herhangi bir okuma sırası var mı anlamadım. test4 dosyasını eklediğimde ilk onu okuyor. Sildiğimde test1.csv' yi okuyor.
// test4 base_path dizininde.
read_csv_folder.show()

+--------------------+
|                 _c0|
+--------------------+
|del1;del2;del3;de...|
|delcol1;delcol1;d...|
|delcol1;delcol1;d...|
|                name|
|               Cevat|
|                John|
|             Natalia|
|               col11|
|               test1|
|               test1|
|               test1|
|                col1|
|                  a1|
|                  b1|
|                  c1|
+--------------------+



csv_folder: String = /home/cevat/workspace/my-apache-spark-notes/scala/projects/read_write_csv/src/csv
read_csv_folder: org.apache.spark.sql.DataFrame = [_c0: string]


In [11]:
// Read Multiple CSV: Multiple paths
val read_multi_files = spark.read
    .csv(
        base_path + "/test2.csv",
        base_path + "/test4.csv",
        base_path + "/test1.csv"
    ) // String* demek birden fazla String ifade girebilirsin demek
// Folder' ın içindeki CSV dosyalarını okutmak gibi. 
// CSV dosyalarının yazılış sırasının bir önemi yok.


/*
    NOT: csv("file1.csv,file2.csv,file3.csv")
*/

read_multi_files.show()

+--------------------+
|                 _c0|
+--------------------+
|del1;del2;del3;de...|
|delcol1;delcol1;d...|
|delcol1;delcol1;d...|
|                name|
|               Cevat|
|                John|
|             Natalia|
|                col1|
|                  a1|
|                  b1|
|                  c1|
+--------------------+



read_multi_files: org.apache.spark.sql.DataFrame = [_c0: string]


In [7]:
// Read CSV file with header: Single CSV file
// Bizim okuduğumuz CSV dosyalarında Header bulunmaktaydı yani kolonların isimleri.
// İlk okumamızda kolon isimleri olarak _c0, _c1 yazmasının sebebi Spark' a CSV dosyasındaki
// ilk satırın header olduğunu belirtmediğimiz için.


val read_csv_with_header: DataFrame = spark
      .read
      .option("header", "true") // Bu option ifadesi ile Spark' a ilk satırın header olduğunu belirtmiş oluyoruz.
      .csv(single_csv_path)

// header option değeri Default olarka false' tur.
// Header option' ı Read/Write işlemlerinin ikisinde de kullanılabilir.
read_csv_with_header.show()

// Daha fazla option için: https://spark.apache.org/docs/latest/sql-data-sources-csv.html#data-source-option

+-------+----------+---+---------+-------+
|   name|   surname|age|     city|country|
+-------+----------+---+---------+-------+
|  Cevat|   Armutlu| 26| Istanbul| Turkey|
|   John|      Eric| 56| New York|America|
|Natalia|Khrushchev|  6|Moskovitz|Russiaa|
+-------+----------+---+---------+-------+



read_csv_with_header: org.apache.spark.sql.DataFrame = [name: string, surname: string ... 3 more fields]


In [8]:
// Read CSV file with header: Multiple CSV file
val read_csv_folder_with_header: DataFrame = spark
      .read
      .option("header", "true")
      .csv(path = csv_folder)
// Böyle bir durumda ilk okuduğu dosyanın ilk satırı header olarak almaktadır.

read_csv_folder_with_header.show()

+----------------------------------+
|del1;del2;del3;del4;del5;del6;del7|
+----------------------------------+
|              delcol1;delcol1;d...|
|              delcol1;delcol1;d...|
|                             Cevat|
|                              John|
|                           Natalia|
|                             test1|
|                             test1|
|                             test1|
|                                a1|
|                                b1|
|                                c1|
+----------------------------------+



read_csv_folder_with_header: org.apache.spark.sql.DataFrame = [del1;del2;del3;del4;del5;del6;del7: string]


In [10]:
// Read CSV file with delimiter
// CSV dosyalarında kolonlar arasındaki değerler `,` ile ayrılır. Lakin bazı dosyalarıda başka
// karakterlerde kullanılabilir.

val read_csv_file_with_delimiter: DataFrame = spark
    .read
    .option("header", "true")
    .option("delimiter", ";") // Bu option ile kolon değerlerinin hangi karakter ile ayrıldığı belirtiliyor.
    .csv(base_path + "/test4.csv") //test4 dosyasında kolonlar `;` ile ayrılıyor.

// Default değer: `,`
// Birden fazla karakter değeri de verilebilir.

read_csv_file_with_delimiter.show()

+-------+-------+-------+-------+-------+-------+-------+
|   del1|   del2|   del3|   del4|   del5|   del6|   del7|
+-------+-------+-------+-------+-------+-------+-------+
|delcol1|delcol1|delcol1|delcol1|delcol1|delcol1|delcol1|
|delcol2|delcol2|delcol2|delcol2|delcol2|delcol2|delcol2|
+-------+-------+-------+-------+-------+-------+-------+



read_csv_file_with_delimiter: org.apache.spark.sql.DataFrame = [del1: string, del2: string ... 5 more fields]


In [11]:
// Read CSV file with sep
// sep ifadesi delimiter ile aynıdır.

val read_csv_file_with_delimiter: DataFrame = spark
    .read
    .option("header", "true")
    .option("sep", ";")
    .csv(base_path + "/test4.csv") //test4 dosyasında kolonlar `;` ile ayrılıyor.
read_csv_file_with_delimiter.show()

+-------+-------+-------+-------+-------+-------+-------+
|   del1|   del2|   del3|   del4|   del5|   del6|   del7|
+-------+-------+-------+-------+-------+-------+-------+
|delcol1|delcol1|delcol1|delcol1|delcol1|delcol1|delcol1|
|delcol2|delcol2|delcol2|delcol2|delcol2|delcol2|delcol2|
+-------+-------+-------+-------+-------+-------+-------+



read_csv_file_with_delimiter: org.apache.spark.sql.DataFrame = [del1: string, del2: string ... 5 more fields]


In [12]:
// Read CSV file with quote
// Bazen CSV dosyalarındaki değerler delimiter karakterini içerebilir.
// Bu gibi durumlarda ise CSV dosyasında bu değer belirli bir karakterin içinde yazılır: "a1, a5" gibi.
// İşte bu gibi değerlerin hangi karakterin içine yazıldığını bu option ile belirliyoruz.

val read_csv_file_with_quote: DataFrame = spark
      .read
      .option("header", "true")
      .option("delimiter", ",")
      .option("quote", "'") // Kolon değeri içinde , kullanmak için değeri 'a1, a5' şeklinde yazdığını belirtiyorsun.
      .csv(base_path + "/test2.csv")

// Default değeri: Çift tırnaktır.
    
read_csv_file_with_quote.show()

+----+----+----+-----+
|col1|col2|col3| col4|
+----+----+----+-----+
|  a1|  a2|  a3|a4,a5|
|  b1|  b2|  b3|   b4|
|  c1|  c2|  c3|   c4|
+----+----+----+-----+



read_csv_file_with_quote: org.apache.spark.sql.DataFrame = [col1: string, col2: string ... 2 more fields]


In [20]:
// Read CSV file with inferSchema

// Bu option ile CSV dosyasındaki kolonların tipi Spark belirliyor.
val read_csv_file_with_inferSchema: DataFrame = spark
      .read
      .option("header", "true")
      .option("delimiter", ",")
      .option("quote", "'")
      .option("inferSchema", "true")
      .csv(base_path + "/test5.csv")
      
read_csv_file_with_inferSchema.printSchema()
read_csv_file_with_inferSchema.show()

// without inferSchema
val read_csv_file_without_inferSchema: DataFrame = spark
      .read
      .option("header", "true")
      .option("delimiter", ",")
      .option("quote", "'")
      .csv(base_path + "/test5.csv")
      
read_csv_file_without_inferSchema.printSchema()
read_csv_file_without_inferSchema.show()

// Bütün kolonları String olarak okudu.

root
 |-- testcol1: integer (nullable = true)
 |-- testcol2: integer (nullable = true)
 |-- testcol3: double (nullable = true)

+--------+--------+--------+
|testcol1|testcol2|testcol3|
+--------+--------+--------+
|     123|      -4|    12.4|
|      32|      -5|    45.6|
|      43|      -6|    67.8|
+--------+--------+--------+

root
 |-- testcol1: string (nullable = true)
 |-- testcol2: string (nullable = true)
 |-- testcol3: string (nullable = true)

+--------+--------+--------+
|testcol1|testcol2|testcol3|
+--------+--------+--------+
|     123|      -4|    12.4|
|      32|      -5|    45.6|
|      43|      -6|    67.8|
+--------+--------+--------+



read_csv_file_with_inferSchema: org.apache.spark.sql.DataFrame = [testcol1: int, testcol2: int ... 1 more field]
read_csv_file_without_inferSchema: org.apache.spark.sql.DataFrame = [testcol1: string, testcol2: string ... 1 more field]


In [47]:
// Read CSv file with timestampFormat
// dateFormat ile çalıştıramadım.

val read_csv_file_with_timestampFormat: DataFrame = spark
  .read
  .option("header", "true")
  .option("inferSchema", "true")
  .option("timestampFormat", "dd-MM-yyyy") // timestamp formatını belirttiğimiz option
  .csv(base_path + "/test6.csv")

// timestamp format değerleri için: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

read_csv_file_with_timestampFormat.show()
read_csv_file_with_timestampFormat.printSchema()

+-----+-------+-------------------+
| name|surname|               date|
+-----+-------+-------------------+
|cevat|armutlu|1995-09-12 00:00:00|
|ahmet| mehmet|1888-04-28 00:00:00|
+-----+-------+-------------------+

root
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- date: timestamp (nullable = true)



read_csv_file_with_timestampFormat: org.apache.spark.sql.DataFrame = [name: string, surname: string ... 1 more field]


In [51]:
// Read CSV file with Schema: 1

val read_csv_file_with_schema: DataFrame = spark
    .read
    .schema("name string, surname string, date date") // Schema bu şekilde belirtilebilir. Kolon adı ve tipi şeklinde.
    .options(Map(
    "header" -> "true",
    "inferSchema" -> "true",
    "delimiter" -> ",",
    "dateFormat" -> "dd-MM-yyyy"
    ))
    .csv(base_path + "/test6.csv")

read_csv_file_with_schema.printSchema()
read_csv_file_with_schema.show()

+-----+-------+-------------------+
| name|surname|               date|
+-----+-------+-------------------+
|cevat|armutlu|1995-09-12 00:00:00|
|ahmet| mehmet|1888-04-28 00:00:00|
+-----+-------+-------------------+

root
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- date: timestamp (nullable = true)



read_csv_file_with_timestampFormat: org.apache.spark.sql.DataFrame = [name: string, surname: string ... 1 more field]


In [54]:
import org.apache.spark.sql.types._

val struct: StructType = StructType(Array(
    StructField(name = "name", dataType = StringType, nullable = false),
    StructField(name = "surname", dataType = StringType, nullable = false),
    StructField(name = "date", dataType = DateType, nullable = false)
))

val df: DataFrame = spark
    .read
    .schema(struct)
    .options(Map(
    "header" -> "true", // true değil "true" olmalı
    "delimiter" -> ",",
    "dateFormat" -> "dd-MM-yyyy",
    "nullValue" -> "cevat" // Hangi değerin null olarak yazılacağı belirtilir. cevat isimli değerler yerine null yazar.
    ))
    .csv(base_path + "/test6.csv")

df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- date: date (nullable = true)

+-----+-------+----------+
| name|surname|      date|
+-----+-------+----------+
| null|armutlu|1995-09-12|
|ahmet| mehmet|1888-04-28|
+-----+-------+----------+



import org.apache.spark.sql.types._
struct: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,false), StructField(surname,StringType,false), StructField(date,DateType,false))
df: org.apache.spark.sql.DataFrame = [name: string, surname: string ... 1 more field]
